![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [3]:
# Importer les bibliothèques nécessaires
from AlgorithmImports import *
import numpy as np
import pandas as pd
import statsmodels.api as sm
from arch.unitroot.cointegration import engle_granger
import seaborn as sns
import matplotlib.pyplot as plt

# Initialiser QuantBook
qb = QuantBook()

# Sélectionner les ETFs sectoriels
sector_etfs = ["XLF", "XLK", "XLE", "XLY", "XLP", "XLI", "XLV", "XLU", "XLRE", "XLC", "XLB"]
symbols = [qb.AddEquity(etf, Resolution.Daily).Symbol for etf in sector_etfs]

# Obtenir les données historiques (2 ans)
history = qb.History(symbols, 500, Resolution.Daily)

# Préparer les données
prices = history.close.unstack(level=0)

# ** Étape 1 : Matrice de corrélation **
# Calculer les rendements quotidiens
daily_returns = prices.pct_change().dropna()

# Calculer la matrice de corrélation
correlation_matrix = daily_returns.corr()

# Afficher la matrice de corrélation sous forme de carte thermique
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", cbar=True)
plt.title("Matrice de corrélation des rendements quotidiens")
plt.show()

# ** Étape 2 : Tester la co-intégration entre deux ETFs **
etf1 = "XLK"  # Exemple ETF 1 (Technologie)
etf2 = "XLY"  # Exemple ETF 2 (Consommation discrétionnaire)

# Extraire les prix des deux ETFs
etf1_prices = prices[etf1].dropna()
etf2_prices = prices[etf2].dropna()

# Vérifier la co-intégration avec Engle-Granger
model = engle_granger(etf1_prices, etf2_prices, trend="n", lags=0)
print(f"P-valeur du test de co-intégration entre {etf1} et {etf2} : {model.pvalue}")

# Calculer les coefficients de co-intégration via une régression
X = sm.add_constant(etf2_prices)
regression = sm.OLS(etf1_prices, X).fit()

# Récupérer les coefficients
beta0 = regression.params[0]  # Constante
beta1 = regression.params[1]  # Coefficient du second ETF

# Calculer le spread
spread = etf1_prices - (beta0 + beta1 * etf2_prices)

# Visualiser les prix et les écarts
plt.figure(figsize=(14, 7))

# Visualisation des prix
plt.subplot(2, 1, 1)
plt.plot(etf1_prices, label=f"Prix de {etf1}")
plt.plot(etf2_prices, label=f"Prix ajusté de {etf2}")
plt.title("Prix des ETFs")
plt.legend()

# Visualisation du spread
plt.subplot(2, 1, 2)
plt.plot(spread, label="Écart (spread)")
plt.axhline(spread.mean(), color='red', linestyle='--', label='Moyenne du spread')
plt.title(f"Spread entre {etf1} et {etf2}")
plt.legend()

plt.tight_layout()
plt.show()


In [4]:
# Tester la co-intégration pour toutes les paires
from itertools import combinations
results = []
for etf1, etf2 in combinations(sector_etfs, 2):
    etf1_prices = prices[etf1].dropna()
    etf2_prices = prices[etf2].dropna()
    if len(etf1_prices) == len(etf2_prices):
        model = engle_granger(etf1_prices, etf2_prices, trend="n", lags=0)
        results.append((etf1, etf2, model.pvalue))
results.sort(key=lambda x: x[2])  # Trier par p-valeur
for etf1, etf2, pval in results[:5]:
    print(f"P-valeur pour {etf1} et {etf2} : {pval}")
